In [ ]:
# Importing the necessary libraries
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import csv
import pandas as pd
from math import ceil
options = Options()
options.headless = True

In [ ]:
# Creates a new .csv file that the data will be written to
csv_file = open('KiaSeltosCarReviews.csv', 'w', encoding="UTF-8", newline="")
writer = csv.writer(csv_file)

# Define the variables (future data frame columns) to be scraped
writer.writerow(['Customer Name' , 'Review text' , 'Review date' , 'Review Star-Rating', 'Review Website'])

#Invoking the webdriver
driver = webdriver.Chrome(options=options, executable_path=r'C:/Users/nisha/Desktop/chromedriver.exe')

In [ ]:
try:
    # Find total number of reviews posted on cardekho.com
    # Each page defaults to showing 30 reviews, so take the ceiling of the total number of job postings divided by 30
    # to get the number of pages
    page="https://www.cardekho.com/kia/seltos/user-reviews"
    driver.get(page)
    time.sleep(5)
    total = driver.find_element_by_xpath('//div[@class ="startRating"]//span[@class="bottomText"]').text
    n = int(ceil(float(total.split()[0])/30))
    print(n)
    k = 1  
    while k <= n: # Initializing k with 1 and looping it till it exhausts all the pages 
        print("Scraping " + "page : " + str(k))  # Printing the page no which is being scraped
        try:
            p = 1
            # Fetching the handle for all the reviews available on the page opened by the driver
            Reviews = driver.find_elements_by_xpath('//div[@class="readReviewBox"]')
            for review in Reviews: # Taking one review at a time to extract the info
                print("Scraping " + "review #" + str(p) + " of page : " + str(k))
                p = p + 1
                reviews_dict = {}       # Creating an empty dictionary by the name "reviews_dict"
                try:
                    #Fetching Reviewer_Name
                    Name = review.find_element_by_xpath('.//div[@class ="authorSummary"]//div[@class = "name"]').text
                    Reviewer_Name = ' '.join(Name.split()[1:])
                except:
                    Reviewer_Name = ""
                try:
                    #First fetching the link where full review text and website name is embedded
                    #Opening the fetched link with the help of driver and then storing the review text
                    Opening_link = review.find_element_by_xpath('.//div[@class ="contentspace"]//h3//a').get_attribute('href')
                    driver1 = webdriver.Chrome(options=options, executable_path=r'C:/Users/nisha/Desktop/chromedriver.exe')
                    driver1.get(Opening_link)
                    time.sleep(3)
                    Review_text = driver1.find_element_by_xpath('.//p[@class ="contentheight"]').text
                except:
                    Review_text = ""
                try:
                    #Fetching the name of website on which the review is posted
                        Review_Website = driver1.title.split()[-1]
                        driver1.close()
                except:
                    Review_Website = ""
                try:
                    #Fetching the review date of the review
                    Date = review.find_element_by_xpath('.//div[@class ="date"]').text
                    Review_date = ''.join([date for date in Date.split()[1:4]])
                except:
                    Review_date = ""
                try:
                    #Fetching the rating provided by the reviewer
                    elem = review.find_elements_by_css_selector("span[class ^= 'stars']")
                    x = {'full-fill' : 1 , 'half-empty' : 0.5 ,'full-empty' : 0 }
                    Review_Star_Rating = sum([x['-'.join(elem[i].get_attribute("class").split()[2].split('-')[-2:])] for i in range(0,5)])
                except:
                    Review_Star_Rating = ""     
                
                # Write the results of the above to a dictionary. Note that each review will have its
                # own dictionary, but all dictionaries for all the rows will all have the same keys. This
                # allows Selenium to write the contents of these dictionaries into a coherent .csv file    
                reviews_dict['Customer Name'] = Reviewer_Name
                reviews_dict['Review text'] = Review_text
                reviews_dict['Review date'] = Review_date
                reviews_dict['Review Star-Rating'] = Review_Star_Rating
                reviews_dict['Review Website'] = Review_Website
                writer.writerow(reviews_dict.values())
                
    
            driver.close()
            k = k+1
            try :
                driver = webdriver.Chrome(options=options, executable_path=r'C:/Users/nisha/Desktop/chromedriver.exe')
                page="https://www.cardekho.com/kia/seltos/user-reviews"  +"/" + str(k) +"?" + "subtab=latest"
                driver.get(page)
                time.sleep(5)
            except:
                break
        # If an error is thrown unrelated to the above variables, print the error to the terminal
        # console, close the .csv file, and break the while loop.
        except Exception as e:
            print(e)
            csv_file.close()
            driver.close()
            break
            
# If an error is thrown unrelated to the above variables, print the error to the terminal
# console, close the .csv file, and break the while loop.           
except Exception as e:
    print(e)
    csv_file.close()
    driver.close()

#Closing the csv file and the driver that has been opened
csv_file.close()
driver.close()

By changing the link we can obtain the reviews data for rest of the two cars.